# Laboratorio 5 - Python en Paralelo
**alumno09:** Laura Llamas López

## Summing all the prime numbers below a given number

In [1]:
# Configuración para ejecución desde línea de comandos
import sys

# Obtener el valor de number desde argumentos de línea de comandos
try:
    if len(sys.argv) > 1:
        number = int(sys.argv[1])
        print(f"Ejecutando con number = {number:,}")
    else:
        number = 2_500_000  # Valor por defecto para ejecución interactiva
        print(f"Ejecutando con number = {number:,} (valor por defecto)")
except (ValueError, IndexError) as e:
    print(f"Error en argumento: {e}")
    print(f"Usando valor por defecto: 2,500,000")
    number = 2_500_000

Error en argumento: invalid literal for int() with base 10: '-f'
Usando valor por defecto: 2,500,000


In [1]:
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

# number = 2_500_000
suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 4.779455184936523
The prime sum below  2500000 is  219697708195  and the time taken is 4.61 s ± 4.59 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


## a) Optimización con Numba

In [1]:
from numba import njit
import time

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

suma = 0
N = 3 # number of loops

# Warmup: pre-compilación JIT de Numba para medir solo el tiempo de ejecución
warm_up = sum_primes(1000)

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken with Numba is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken with Numba is", tiempo)

The prime sum below  2500000 is  219697708195  and the time taken with Numba is 0.5345462958017985
The prime sum below  2500000 is  219697708195  and the time taken with Numba is 578 ms ± 1.18 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


### Interpretación de resultados

**Código original (sin Numba):**
- Método con `map()`: 4.779 segundos
- Método con `%timeit`: 4.61 s ± 4.59 ms

**Código con Numba (@njit):**
- Método con `map()`: 0.535 segundos
- Método con `%timeit`: 578 ms ± 1.18 ms

### Mejora obtenida
- **Método map():** 4.779 / 0.535 = **8.93x**
- **Método %timeit:** 4.61 / 0.578 = **7.98x**

La optimización con Numba mediante el decorador `@njit` ha proporcionado una mejora muy significativa, cercana al factor 10x esperado por el enunciado, demostrando la efectividad de Numba en funciones con cálculos intensivos como la verificación de números primos.



## b) Multiprocessing con Pool

In [1]:
from numba import njit
from multiprocessing import Pool
import time

# Mantenemos la función optimizada con Numba del ejercicio a)
@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

# Función para calcular la suma de primos en un rango [start, end)
def sum_primes_range(args):
    start, end = args
    result = 0
    for i in range(start, end):
        result += if_prime(i)
    return result

# Función principal con paralelización usando Pool
def sum_primes_parallel(number, num_processes=4):
    chunk_size = number // num_processes
    ranges = []
    for p in range(num_processes):
        start = p * chunk_size
        end = (p + 1) * chunk_size if p < num_processes - 1 else number
        ranges.append((start, end))

    # Creamos el pool y ejecutamos en paralelo
    with Pool(processes=num_processes) as pool:
        partial_sums = pool.map(sum_primes_range, ranges)

    # Reducimos sumando los resultados parciales
    return sum(partial_sums)

# Warmup: pre-compilación JIT completa
_ = sum_primes_range((0, 1000))

# Ejecutar y medir tiempo con 4 núcleos
if len(sys.argv) > 2:
    num_processes = int(sys.argv[2])
else:
    num_processes = 4
N = 3  # número de repeticiones para el promedio

start = time.time()
for i in range(N):
    suma = sum_primes_parallel(number, num_processes)
stop = time.time()
tiempo = (stop - start) / N

print(f"The prime sum below {number} with multiprocessing (Pool, {num_processes} cores) is {suma} and the time taken is {tiempo} seconds")

The prime sum below 2500000 with multiprocessing (Pool, 4 cores) is 219697708195 and the time taken is 0.18040982882181802 seconds


### Interpretación de resultados

- **Tiempo con multiprocessing (Pool, 4 cores):** 0.1804 segundos
- **Suma de primos:** 219697708195 ✓

La paralelización con multiprocessing y Pool ha conseguido una **mejora adicional de 3x** sobre el código ya optimizado con Numba, aprovechando los 4 núcleos de la CPU. La eficiencia de paralelización es del **74.3%** (2.97/4), lo que indica una distribución efectiva del trabajo con overhead mínimo. El código es ahora **26.5x más rápido** que la versión original.

## c) Numba con prange

In [2]:
from numba import njit, prange, set_num_threads
import time

# Configurar el número de threads de Numba
if len(sys.argv) > 2:
    num_threads = int(sys.argv[2])
else:
    num_threads = 4

set_num_threads(num_threads)

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

# Función paralelizada con prange
@njit(parallel=True)
def sum_primes_prange(x):
    result = 0
    for i in prange(x):  # prange en lugar de range
        result += if_prime(i)
    return result

# Warmup: pre-compilación JIT completa
_ = sum_primes_prange(1000)

N = 3  # número de repeticiones para el promedio

start = time.time()
for i in range(N):
    suma = sum_primes_prange(number)
stop = time.time()
tiempo = (stop - start) / N

print(f"The prime sum below {number} with Numba prange (4 threads) is {suma} and the time taken is {tiempo} seconds")

The prime sum below 2500000 with Numba prange (4 threads) is 219697708195 and the time taken is 0.07092054684956868 seconds


## Interpretación de resultados

- **Tiempo con Numba prange (4 núcleos):** 0.0709 segundos
- **Suma de primos:** 219697708195 ✓

La paralelización con **Numba prange** ha resultado ser **la más eficiente**, siendo **2.5x y 7.5x más rápida** que Multiprocessing + Pool y Numba secuencial, respectivamente. Esto se debe a que prange opera con **núcleos dentro del mismo proceso**, evitando el overhead de crear múltiples procesos y la serialización de datos que requiere multiprocessing. El código con Numba prange es ahora **67.4x más rápido** que la versión original, siendo la solución óptima para este problema.

## d) Resultados de ejecución en cola mendel

### Para 10^6

### Para 10^7

## e) Comparación e interpretación de los tiempos obtenidos para los cuatro procedimientos

### Resultados para number = 10^6 

| Método | 1 core | 2 cores | 4 cores | 8 cores |
|------------------------|--------:|--------:|--------:|--------:|
| Código original        |  2.26   |  2.26   |  2.29   |  2.29   |
| Numba secuencial       |  0.316  |  0.315  |  0.315  |  0.317  |
| Multiprocessing (Pool) |  0.371  |  0.207  |  0.123  |  0.090  |
| Numba prange (threads) |  0.115  |  0.071  |  0.052  |  0.027  |

### Resultados para number = 10^7 

| Método | 1 core | 2 cores | 4 cores | 8 cores |
|------------------------|--------:|--------:|--------:|--------:|
| Código original        | 59.19   | 59.24   | 60.11   | 60.03   |
| Numba secuencial       |  4.55   |  4.56   |  4.54   |  4.56   |
| Multiprocessing (Pool) |  4.88   |  2.75   |  1.46   |  0.76   |
| Numba prange (threads) |  2.87   |  1.79   |  0.96   |  0.50   |

*Nota: Todos los tiempos están expresados en segundos.*

### Interpretación y análisis de resultados

Los resultados del código original y Numba secuencial tienen carácter ilustrativo en este análisis, ya que estos métodos no implementan paralelización con múltiples núcleos, por lo que sus tiempos permanecen prácticamente constantes independientemente del número de cores especificados. Sin embargo, resulta interesante observar cómo al multiplicar por diez el tamaño del problema (pasando de 10^6 a 10^7), el tiempo se incrementa de forma proporcional: en el código original aumenta de 2.26 segundos a prácticamente un minuto, mientras que con Numba secuencial pasa de 0.316 a 4.55 segundos.

Al examinar los métodos que sí utilizan paralelización, se aprecia claramente el beneficio de distribuir la carga computacional entre varios núcleos. Con multiprocessing, observamos reducciones de tiempo progresivas conforme aumenta el número de cores: para el problema pequeño (10^6) pasamos de 0.371 segundos con un solo core a 0.090 segundos con ocho cores, mientras que para el problema grande (10^7) la reducción es aún más notable, desde 4.88 segundos hasta 0.76 segundos. Esta mayor eficiencia con datasets más grandes se explica porque el overhead asociado a la creación y gestión de procesos se amortiza mejor cuando hay más trabajo computacional que realizar.

Numba prange se revela como el método más eficiente en todos los escenarios evaluados. Para el problema de 10^6 elementos, consigue reducir el tiempo desde 0.115 segundos (un core) hasta apenas 0.027 segundos utilizando ocho cores. Con 10^7 elementos, la mejora es igualmente impresionante: de 2.87 segundos baja a medio segundo. Lo más destacable es que Numba prange supera consistentemente a multiprocessing, siendo entre 1.5 y 3.3 veces más rápido cuando ambos utilizan ocho cores. Esta ventaja se debe a que los threads que emplea Numba tienen un overhead menor que los procesos de multiprocessing y no necesitan serializar datos para comunicarse entre sí.

Si comparamos el mejor resultado obtenido (Numba prange con ocho cores) contra el punto de partida (código Python original), las mejoras son verdaderamente significativas: aproximadamente 84 veces más rápido para 10^6 elementos y 120 veces más rápido para 10^7. Esto demuestra que combinar compilación JIT con paralelización mediante threads resulta extremadamente efectivo para problemas intensivos en cálculo como el análisis de números primos, haciendo viable procesar volúmenes de datos mucho mayores en tiempos razonables.